In [3]:
import numpy as np
import random
import pandas as pd
from scipy.optimize import minimize
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io as sio
import statsmodels.api as sm


# Important Note

### purpose of this file is to get parameters (alphaG, alphaL, beta) of Q learning for real behavioral data

### reward in this file is always subjective reward, i.e. 0 <==> the rat did not get reward

## Load Data

In [4]:
day1 = pd.read_csv('/Users/erika/Data/PRL/behavioral_data/Day 1.csvQ_Analysis.csv')
day21 = pd.read_csv('/Users/erika/Data/PRL/behavioral_data/Day 21.csvQ_Analysis.csv')

In [27]:
day1.columns

Index(['Unnamed: 0', 'subject', 'prog', 'target_lat', 'non_target_lat', 'ITI',
       'ISI', 'Tone', 'TargetOutcome', 'NonTargetOutcome', 'NAN', 'TargetSide',
       'Omission', 'TimeOut', 'SessionTime', 'NAN.1', 'NAN.2',
       'SwitchContingency', 'trial', 'position'],
      dtype='object')

In [48]:
# Let rat 30 be chosen
# low tone <==> reward
day1_30 = day1[day1['subject']==30]
day21_30 = day21[day21['subject']==30]

In [49]:
day1_30 = day1_30[['Tone','TargetOutcome','NonTargetOutcome','TargetSide']]
day21_30 = day21_30[['Tone','TargetOutcome','NonTargetOutcome','TargetSide']]

In [72]:
day1_30[:10]

,Tone,TargetOutcome,NonTargetOutcome,TargetSide
291,2000.0,1.0,0.0,1.0
292,5000.0,0.0,-1.0,1.0
293,5000.0,0.0,-1.0,1.0
294,2000.0,0.0,1.0,1.0
295,5000.0,-1.0,0.0,1.0
296,5000.0,0.0,-1.0,1.0
297,2000.0,0.0,1.0,1.0
298,5000.0,0.0,-1.0,1.0
299,5000.0,0.0,-1.0,1.0
300,5000.0,0.0,-1.0,1.0


## Get Actions and Rewards

In [73]:
1.0 ==1

True

In [74]:
#df: 4 columns of 'Tone','TargetOutcome','NonTargetOutcome','TargetSide'; df is for one rat in one session
# consisting of 1(left) and 2(right)
# suppose TargetSide=1.0 means left, 4.0 means right
def get_action(df):
    actions = []
    targetSide = df['TargetSide'].tolist()
    targetOut = df['TargetOutcome'].tolist()
    for i in range(df.shape[0]):
        act = -1
        if(targetSide[i]==1.0):
            if(targetOut[i]==0):
                act = 2
            else:
                act = 1
        else:
            if(targetOut[i]==0):
                act = 1
            else:
                act = 2
        actions.append(act)
        
    return actions
    

In [77]:
# df: 4 columns of 'Tone','TargetOutcome','NonTargetOutcome','TargetSide'; df is for one rat in one session
# reward is subjective, depending on whether the rat got reward
def get_reward(df):
    rewards = []
    targetOut = df['TargetOutcome'].tolist()
    nontargetOut = df['NonTargetOutcome'].tolist()
    
    for i in range(df.shape[0]):
        rew = -1
        if(targetOut[i] == 1 or nontargetOut[i] == 1):
            rew = 1
        else:
            rew = 0
        rewards.append(rew)
        
    return rewards

## MLE Functions

In [260]:
# action is from choice_Log consisting of 1 and 2s
# reward is subjective, depending on whether the rat got reward
def neg_log_likelihood_2Q_2alpha(alphaG,alphaL,beta,actions,rewards,Q=[0.1,0.1],gamma=0,Qleft=True): 
    n = len(actions)
    sum_ll = 0
    for i in range(n):
        turn = int(actions[i] - 1)
        rew = int(rewards[i])
        
        if int(rew) == 1: # alpha_gain
            Q[turn] = Q[turn] + alphaG*(rew - Q[turn] + gamma*np.max(Q))
        elif int(rew) == 0: # alpha_loss
            Q[turn] = Q[turn] + alphaL*(rew - Q[turn] + gamma*np.max(Q))
        
        temp = 1/(np.exp(0-beta*(Q[0])) + 1)
        
        if int(turn) == 0:
            prob = temp
        else:
            prob = 1 - temp
        
        sum_ll = sum_ll - np.log(prob + np.exp(0-8)) # add a smoother to avoid warnings
    
    return sum_ll/50 #rescale

In [261]:
# params = [alphaG0,alphaL0,beta0]
# args = [actions,rewards]
def helper_func_2alpha(params,args):
    alphaG0 = params[0]
    alphaL0 = params[1]
    beta0 = params[2]
    actions = args[0]
    rewards = args[1]
    
    sum_ll = neg_log_likelihood_2Q_2alpha(alphaG0,alphaL0,beta0,actions,rewards)
    
    return sum_ll

In [262]:
# function that estimates the maximum-likelihood beta_hat numerically
# parameters: actions, a numpy array recording action of agent in each turn; beta is the parameter in density func
# return minimization summary and print beta_hat
def MLE_2alpha(actions,rewards,alphaG0,alphaL0,beta0):
    initial_guess = [alphaG0,alphaL0,beta0]
    args = [actions,rewards]
    bounds = ((0,1),(0,1),(0,20))
    result = minimize(helper_func_2alpha,initial_guess,args=args,bounds = bounds)
    if(result.success):
        #print(result.message)
        #print('The MLE for beta is', result.x)
        #print('Iteration =', result.nit)
        a=0
    else:
        print('The optimization did not converge, beta0 equals', beta0,', and alphaG0 equals',alphaG0)
    return result
    
    

## Test

In [146]:
df = pd.read_csv('/Users/erika/Data/PRL/behavioral_data/UH_RL_rats.csv')
df.head()

,trial,TargetOffset,NonTargetOffset,ITI,ISI,Tone,TargetReward,NonTargetReward,RewardLat,Target Lever,...,response,feedback,state,action,reward,alpha_gain,alpha_loss,beta,Q,PE
0,1,31507,0,0,0,0,-1,0,0,0,...,rich,0.0,1.0,1.0,0.0,0.139603,1.0,20.0,0.000000,-0.500000
1,2,32538,0,0,0,0,1,0,0,0,...,rich,1.0,1.0,1.0,1.0,0.139603,1.0,20.0,0.139603,1.000000
2,3,0,33594,0,0,0,0,-1,0,0,...,lean,0.0,1.0,2.0,0.0,0.139603,1.0,20.0,0.000000,-0.500000
3,4,34626,0,0,0,0,1,0,0,0,...,rich,1.0,1.0,1.0,1.0,0.139603,1.0,20.0,0.259718,0.860397
4,5,0,35533,0,0,0,0,-1,0,0,...,lean,0.0,1.0,2.0,0.0,0.139603,1.0,20.0,0.000000,0.000000


In [147]:
rats = df['subject'].unique()

In [148]:
# divide df into 5 dfs according to session
# '1,11,16,6,Best'
session = df['session'].unique()
df_1 = df[df['session']=='1']
df_11 = df[df['session']=='11']
df_16 = df[df['session']=='16']
df_6 = df[df['session']=='6']
df_best = df[df['session']=='Best']
df_list = [df_1,df_11,df_16,df_6,df_best]

In [211]:
# Let rat 10 and session 'Best' be chosen
df_10_best = df_best[df_best['subject']==10]
df_10_best = df_10_best[['Tone','TargetReward','NonTargetReward','Target Lever','lever','response','feedback','state','action','reward','alpha_gain','alpha_loss','beta','PE','Q']]

In [236]:
df_19_best = df_best[df_best['subject']==19]
df_19_best = df_19_best[['Tone','TargetReward','NonTargetReward','Target Lever','lever','response','feedback','state','action','reward','alpha_gain','alpha_loss','beta','PE','Q']]

In [187]:
# Dan's parameters
df_10_best[['alpha_gain','alpha_loss','beta']].head()

,alpha_gain,alpha_loss,beta
888,0.480102,0.61508,0.355195
889,0.480102,0.61508,0.355195
890,0.480102,0.61508,0.355195
891,0.480102,0.61508,0.355195
892,0.480102,0.61508,0.355195


In [199]:
action_10_best = df_10_best['action'].tolist()
reward_10_best = df_10_best['reward'].tolist()

In [237]:
action_19_best = df_19_best['action'].tolist()
reward_19_best = df_19_best['reward'].tolist()

In [251]:
def my_minimize(actions,rewards, alphaG=0, alphaL=0, beta=0):
    likelihoods = []
    #step = 1/10  #1/int(iteration**(1/3))
    alphaG_arr = np.arange(0,1,0.05)
    alphaL_arr = np.arange(0,1,0.05)
    beta_arr = np.arange(0,20,0.5)
    num_G = len(alphaG_arr)
    num_L = len(alphaL_arr)
    num_b = len(beta_arr)
    
    index_G = 0
    index_L = 0
    index_b = 0
    
    temp = 100000
    
    for i in range(num_G):
        for j in range(num_L):
            for k in range(num_b):
                
                ll = neg_log_likelihood_2Q_2alpha(alphaG_arr[i],alphaL_arr[j],beta_arr[k],actions,rewards)
                likelihoods.append(ll)
                
                if(ll < temp):
                    temp = ll
                    index_G = i
                    index_L = j
                    index_b = k
    
    alphaG_ = alphaG_arr[index_G]
    alphaL_ = alphaL_arr[index_L]
    beta_ = beta_arr[index_b]
    
    return [alphaG_,alphaL_,beta_]

In [238]:
# dan's param
df_19_best[['alpha_gain','alpha_loss','beta']].head()

,alpha_gain,alpha_loss,beta
13815,0.354138,1.0,0.37386
13816,0.354138,1.0,0.37386
13817,0.354138,1.0,0.37386
13818,0.354138,1.0,0.37386
13819,0.354138,1.0,0.37386


In [263]:
# difference between likelihood
dan_ll = neg_log_likelihood_2Q_2alpha(0.3541,1.0,0.37386,action_19_best,reward_19_best)
my_ll = neg_log_likelihood_2Q_2alpha(0.4,0.95,3.5,action_19_best,reward_19_best)

In [264]:
dan_ll

3.9886884398573925

In [265]:
my_ll

3.9165242887875804

In [268]:
my_minimize(action_19_best,reward_19_best)

[0.25, 0.9500000000000001, 2.5]

In [266]:
sum(reward_19_best)

181.0

In [267]:
init_guess = [0.3,0.6,5] #alphaG,alphaL,beta
results = MLE_2alpha(action_10_best,reward_10_best,init_guess[0],init_guess[1],init_guess[2])
results

The optimization did not converge, beta0 equals 5 , and alphaG0 equals 0.3


      fun: 4.9451871395166265
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([28790647.91391581, 28790640.05848111, 28790643.51350085])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 84
      nit: 0
   status: 2
  success: False
        x: array([0.3, 0.6, 5. ])

In [207]:
neg_log_likelihood_2Q_2alpha(0.3,0.6,5,action_10_best,reward_10_best)

2.7479854198183147

## MLE, Rat 30 Day 1

In [79]:
actions_30 = get_action(day1_30)
rewards_30 = get_reward(day1_30)

In [142]:
init_guess = [0.5,0.5,3] #alphaG,alphaL,beta
results = MLE_2alpha(actions_30,rewards_30,init_guess[0],init_guess[1],init_guess[2])

The optimization did not converge, beta0 equals 3 , and alphaG0 equals 0.5


In [143]:
results

      fun: 296.43649600043045
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([58255829.69403058, 58255652.05233829, 58255779.91605542])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 84
      nit: 0
   status: 2
  success: False
        x: array([0.5, 0.5, 3. ])

## Statistical Tests

In [ ]:
# supoose we now have several rats' day1 and day 21 parameters(alphaG,alphaL,beta); paired
param_day1 = []
param_day21 = []


In [ ]:
# Q-Q plot to check normality for day 1
for i in range(len(param_day1)):
    stats.probplot(param_day1[i], dist='norm', plot=plt.plot)
    

In [ ]:
# Q-Q plot to check normality for day 21
for i in range(len(param_day21)):
    stats.probplot(param_day21[i], dist='norm', plot=plt.plot)
    

In [ ]:
# Whelch's t-test of the mean of params in day 1 versus day 21
# null hypothesis: params in day1 = day21 
t_list = []
p_list = []
for i in range(len(param_day1)):
    equal_variance = False
    t_stat, p_val = stats.ttest_ind(param_day1[i],param_day21[i], equal_var=equal_variance)
    t_list.append(t_stat)
    p_list.append(p_val)

In [30]:
#bootstrap t-test


In [ ]:
# permutation